## sklearn으로 하이퍼파라미터 튜닝🔥
## GridSearchCV , RandomSearchCV

https://www.dacon.io/codeshare/4568

- 하이퍼파라미터
    - 사용자의 입력값, 즉 우리가 설정 가능한 입력값
    - 사용할 데이터에 따라 가장 적합한 모델, 그리고 그 모델의 하이퍼 파라미터값이 다름
    - 좋은 파라미터 입력값을 찾아주는 작업 => 튜닝
        - GridSearchCV
            - 목표로 하는 하이퍼 파라미터별로 리스트를 만들어 딕셔너리 형태로 파라미터 셋을 입력

In [36]:
import sklearn
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [15]:
iris=load_iris()

In [16]:
label=iris.target
data=iris.data

In [17]:
# train & test data 분리
x_train, x_test, y_train, y_test=train_test_split(iris.data,
                                                 iris.target,
                                                 test_size=0.2,
                                                 random_state=1)

In [18]:
# DecisionTreeClassifier()
dt_clf=DecisionTreeClassifier(random_state=1)

In [19]:
# DecisionTreeClassifier().fit
dt_clf.fit(x_train, y_train)

DecisionTreeClassifier(random_state=1)

In [22]:
# DecisionTreeClassifier().fit().predict()
predict=dt_clf.predict(x_test)

In [24]:
# accaccuracy_score(답안지, 예측값)
accuracy=accuracy_score(y_test, predict)
print('예측 정확도 : {0:.4f}'.format(accuracy))

예측 정확도 : 0.9667


In [32]:
# 의사결정나무의 하이퍼 파라미터 출력
print(f'dt_clf의 하이퍼 파라미터 => {dt_clf.get_params()}')

dt_clf의 하이퍼 파라미터 => {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 1, 'splitter': 'best'}


###### 의사결정나무에서 최적의 값을 찾아낼 파라매터들

*criterion* : 분할 성능 측정 기능

*min_samples_split* : 노드를 분할하기 위한 최소한의 샘플 데이터수로, 과적합을 제어하는데 주로 사용함. 작게 설정할 수록 분할 노드가 많아져 과적합 가능성이 높아짐.

*max_depth* : 트리의 최대 깊이, 깊이가 깊어지면 과적합될 수 있음.

*max_features* : 최적의 분할을 위해 고려할 최대 feature 개수 (default = *None* : 데이터 세트의 모든 피처를 사용)

*min_samples_leaf* : 리프노드가 되기 위해 필요한 최소한의 샘플 데이터수 (과적합 제어 용도), 작게 설정 필요

*max_leaf_nodes* : 리프노드의 최대 개수

In [33]:
# 파라미터별로 리스트를 만들고,
# 파라미터 세트를 딕셔너리로 만들기
param_grid = {
    'criterion':['gini','entropy'], 
    'max_depth':[None,2,3,4,5,6], 
    'max_leaf_nodes':[None,2,3,4,5,6,7], 
    'min_samples_split':[2,3,4,5,6], 
    'min_samples_leaf':[1,2,3], 
    'max_features':[None,'sqrt','log2',3,4,5]
    }

### GridSearchCV 의 인자들

estimator : 보통 알고리즘을 객체로 만들어 넣어준다.

param_grid : 튜닝을 위한 대상 파라미터, 사용될 파라미터를 딕셔너리 형태로 넣어준다.

scoring : 예측 성능을 측정할 평가 방법을 넣는다. 분류 알고리즘일 때는, 'accuracy', 'f1', 회귀 알고리즘일 때는 'neg_mean_squared_error', 'r2' 등을 넣을 수 있다.

cv : 교차 검증에서 몇개로 분할되는지 지정한다.(정수로 넣어주면 K겹 교차검증이 되고, KFold(k) 이런식으로 넣어주어도 무방 // default 값은 cv=3)

refit : True로 하면 최적의 하이퍼 파라미터를 찾아서 estimator를 재학습시킨다. (default 값이 True임)

In [34]:
grid_search=GridSearchCV(dt_clf, param_grid=param_grid, cv=5,
                        scoring='accuracy', refit=True)

In [37]:
grid_search.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=1),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 2, 3, 4, 5, 6],
                         'max_features': [None, 'sqrt', 'log2', 3, 4, 5],
                         'max_leaf_nodes': [None, 2, 3, 4, 5, 6, 7],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [2, 3, 4, 5, 6]},
             scoring='accuracy')

#### 가장 최적 하이퍼 파라미터들의 값, 최고 평균 정확도 수치를 확인

GridSearchCV객체.best_params_ : 최적 하이퍼 파라미터 조합

GridSearchCV객체.best_score_ : 최고 평균 정확도 수치

In [38]:
print('최적의 하이퍼 파라미터 조합 : ', grid_search.best_params_)
print('최고 평균 정확도 수치 : ', grid_search.best_score_)

최적의 하이퍼 파라미터 조합 :  {'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
최고 평균 정확도 수치 :  0.9666666666666668


GridSearchCV는 입력한 하이퍼 파라미터값들을 순차적으로 변경하며 학습과 평가를 수행함.

그리고 cv_results_ 라는 속성에 딕셔너리의 기록하게 되는데, 좀 더 편하게 보기위헤 데이터 프레임으로 변경

In [51]:
cv_resultsDF=pd.DataFrame(grid_search.cv_results_)
cv_resultsDF.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_max_features,param_max_leaf_nodes,param_min_samples_leaf,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001196,0.000401,0.000198,0.000397,gini,None,None,None,1,2,"{'criterion': 'gini', 'max_depth': None, 'max_...",0.958333,0.958333,1.000000,0.958333,0.958333,0.966667,0.016667,1
1,0.001190,0.000745,0.000598,0.000488,gini,None,None,None,1,3,"{'criterion': 'gini', 'max_depth': None, 'max_...",0.958333,0.958333,1.000000,0.958333,0.875000,0.950000,0.040825,86
2,0.000804,0.000402,0.000200,0.000399,gini,None,None,None,1,4,"{'criterion': 'gini', 'max_depth': None, 'max_...",0.958333,0.958333,1.000000,0.958333,0.875000,0.950000,0.040825,86
3,0.000903,0.000658,0.000000,0.000000,gini,None,None,None,1,5,"{'criterion': 'gini', 'max_depth': None, 'max_...",0.958333,0.958333,1.000000,0.958333,0.875000,0.950000,0.040825,86
4,0.000592,0.000484,0.000200,0.000399,gini,None,None,None,1,6,"{'criterion': 'gini', 'max_depth': None, 'max_...",0.958333,0.958333,0.916667,0.958333,0.875000,0.933333,0.033333,334


##### 하이퍼 파라미터의 값이 변화함에 따라 테스트 스코어가 달라짐

GridSearchCV객체.best_estimator_는 GridSearchCV의 refit으로 이미 학습이 된 estimator를 반환함.

따라서 best_estimator_는 이미 최적 하이퍼 파라미터를 적용하여 학습이 된 상태

In [43]:
estimator=grid_search.best_estimator_

In [46]:
pred=estimator.predict(x_test)

In [48]:
print('score : ', accuracy_score(y_test, pred))

score :  0.9666666666666667


In [49]:
# 단점 : 시간 소요가 큼

---
## RandomizedSearchCV
- GridSearch 와 동일한 방식으로 사용하지만 모든 조합을 다 시도하지는 않고, 각 반복마다 임의의 값만 대입해 지정한 횟수만큼 평가
- 몇 번 학습과 평가를 반복할 것인지 시도의 수를 우리가 설정할 수 있기 때문에 비교적 시간이 적게 걸림

- RandomizedSearchCV 가 사용하는 인자들은 GridSearchCV와 거의 동일하지만, n_iter 라는 인자가 추가, param_grid 대신 param_distributions 를 사용

- param_distributions : 튜닝을 위한 대상 파라미터, 사용될 파라미터를 딕셔너리 형태로 넣어준다.

- n_iter : 파라미터 검색 횟수

- n_iter 의 default는 10, RandomizedSearchCV 는 우리가 n_iter에 지정한 횟수만큼만 조합을 반복하여 평가

In [52]:
from sklearn.model_selection import RandomizedSearchCV

dt_clf = DecisionTreeClassifier(random_state=1)

param_dist = {
    'criterion':['gini','entropy'], 
    'max_depth':[None,2,3,4,5,6], 
    'max_leaf_nodes':[None,2,3,4,5,6,7], 
    'min_samples_split':[2,3,4,5,6], 
    'min_samples_leaf':[1,2,3], 
    'max_features':[None,'sqrt','log2',3,4,5]
    }

In [54]:
rand_search = RandomizedSearchCV(dt_clf, param_distributions = param_dist, n_iter = 50, cv = 5, scoring = 'accuracy', refit=True)
rand_search.fit(x_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=1),
                   n_iter=50,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [None, 2, 3, 4, 5, 6],
                                        'max_features': [None, 'sqrt', 'log2',
                                                         3, 4, 5],
                                        'max_leaf_nodes': [None, 2, 3, 4, 5, 6,
                                                           7],
                                        'min_samples_leaf': [1, 2, 3],
                                        'min_samples_split': [2, 3, 4, 5, 6]},
                   scoring='accuracy')

In [55]:
print('best parameters : ', rand_search.best_params_)
print('best score : ', round(rand_search.best_score_, 4))

best parameters :  {'min_samples_split': 4, 'min_samples_leaf': 1, 'max_leaf_nodes': 6, 'max_features': None, 'max_depth': 6, 'criterion': 'gini'}
best score :  0.95


In [56]:
df = pd.DataFrame(rand_search.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_samples_split,param_min_samples_leaf,param_max_leaf_nodes,param_max_features,param_max_depth,param_criterion,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000809,0.000405,0.000000,0.000000,3,2,3,5,None,entropy,"{'min_samples_split': 3, 'min_samples_leaf': 2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,50
1,0.000797,0.000745,0.000399,0.000489,5,2,3,4,2,entropy,"{'min_samples_split': 5, 'min_samples_leaf': 2...",0.958333,0.875000,0.916667,0.958333,0.875000,0.916667,0.037268,24
2,0.000999,0.000004,0.000398,0.000488,6,1,7,sqrt,3,gini,"{'min_samples_split': 6, 'min_samples_leaf': 1...",0.916667,0.875000,0.916667,0.916667,0.916667,0.908333,0.016667,37
3,0.001194,0.000399,0.000000,0.000000,3,3,4,3,None,entropy,"{'min_samples_split': 3, 'min_samples_leaf': 3...",0.958333,0.958333,0.916667,0.958333,0.875000,0.933333,0.033333,2
4,0.001002,0.000006,0.000599,0.000489,3,2,5,3,6,gini,"{'min_samples_split': 3, 'min_samples_leaf': 2...",0.958333,0.958333,0.916667,0.958333,0.875000,0.933333,0.033333,2
5,0.000801,0.000401,0.000399,0.000488,2,1,2,None,5,gini,"{'min_samples_split': 2, 'min_samples_leaf': 1...",0.666667,0.666667,0.708333,0.708333,0.708333,0.691667,0.020412,41
6,0.000993,0.000012,0.000000,0.000000,4,2,2,3,5,gini,"{'min_samples_split': 4, 'min_samples_leaf': 2...",0.666667,0.666667,0.708333,0.708333,0.708333,0.691667,0.020412,41
7,0.000593,0.000795,0.000398,0.000488,5,3,2,None,None,entropy,"{'min_samples_split': 5, 'min_samples_leaf': 3...",0.666667,0.666667,0.708333,0.708333,0.708333,0.691667,0.020412,41
8,0.000592,0.000787,0.000394,0.000483,2,1,5,None,None,entropy,"{'min_samples_split': 2, 'min_samples_leaf': 1...",0.958333,0.958333,0.916667,0.958333,0.875000,0.933333,0.033333,2
9,0.000798,0.000399,0.000200,0.000400,4,1,6,None,6,gini,"{'min_samples_split': 4, 'min_samples_leaf': 1...",0.958333,0.958333,1.000000,0.958333,0.875000,0.950000,0.040825,1
